<a href="https://colab.research.google.com/github/amit51910636/DataScience/blob/main/Confluent_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 20.8 MB/s eta 0:00:00


In [ ]:
# Required connection configs for Kafka producer, consumer, and admin
bootstrap.servers=pkc-619z3.us-east1.gcp.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=BVDQLUUYCEQ2B36K
sasl.password=wKsvL2/e4Vu260owTCH8+z2f6BbRp6K5ylPPgXHNXwPtrNGpFYWh7nOtJ8PmsRnx

# Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-python-client-37d62a4e-b48b-4f3c-ac2c-761be0dd52df


SyntaxError: invalid decimal literal (<ipython-input-2-1000ead08a48>, line 2)

In [ ]:
from confluent_kafka import Producer, Consumer

def read_config():
  # reads the client configuration from client.properties
  # and returns it as a key-value map
  config = {}
  with open("client.properties") as fh:
    for line in fh:
      line = line.strip()
      if len(line) != 0 and line[0] != "#":
        parameter, value = line.strip().split('=', 1)
        config[parameter] = value.strip()
  return config

def produce(topic, config):
  # creates a new producer instance
  producer = Producer(config)

  # produces a sample message
  key = "key"
  value = "value"
  producer.produce(topic, key=key, value=value)
  print(f"Produced message to topic {topic}: key = {key:12} value = {value:12}")

  # send any outstanding or buffered messages to the Kafka broker
  producer.flush()

def consume(topic, config):
  # sets the consumer group ID and offset
  config["group.id"] = "python-group-1"
  config["auto.offset.reset"] = "earliest"

  # creates a new consumer instance
  consumer = Consumer(config)

  # subscribes to the specified topic
  consumer.subscribe([topic])

  try:
    while True:
      # consumer polls the topic and prints any incoming messages
      msg = consumer.poll(1.0)
      if msg is not None and msg.error() is None:
        key = msg.key().decode("utf-8")
        value = msg.value().decode("utf-8")
        print(f"Consumed message from topic {topic}: key = {key:12} value = {value:12}")
  except KeyboardInterrupt:
    pass
  finally:
    # closes the consumer connection
    consumer.close()

def main():
  config = read_config()
  topic = "topic_0"

  produce(topic, config)
  consume(topic, config)


main()

Produced message to topic topic_0: key = key          value = value       
Consumed message from topic topic_0: key = key          value = value       
Consumed message from topic topic_0: key = 123          value = {}          


In [ ]:
from confluent_kafka import Producer

# Required connection configs for Kafka producer, consumer, and admin
conf = {
    'bootstrap.servers': 'pkc-619z3.us-east1.gcp.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'BVDQLUUYCEQ2B36K',
    'sasl.password': 'wKsvL2/e4Vu260owTCH8+z2f6BbRp6K5ylPPgXHNXwPtrNGpFYWh7nOtJ8PmsRnx',
    'session.timeout.ms': 45000,  # Best practice for higher availability
    'client.id': 'ccloud-python-client-37d62a4e-b48b-4f3c-ac2c-761be0dd52df'
}

producer = Producer(conf)

# Example message delivery callback
def delivery_callback(err, msg):
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()}[{msg.partition()}]")

# Produce messages
topic = "topic_0"  # Replace with your topic name
message = "Hello, Kafka User!"

producer.produce(topic, message.encode('utf-8'), callback=delivery_callback)

# Wait for any outstanding messages to be delivered and delivery reports to be received
producer.flush()


Message delivered to topic_0[2]


0

In [ ]:
# prompt: can you now give me the code to consume the message from above kafka cluster

from confluent_kafka import Consumer

conf = {
    'bootstrap.servers': 'pkc-619z3.us-east1.gcp.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'BVDQLUUYCEQ2B36K',
    'sasl.password': 'wKsvL2/e4Vu260owTCH8+z2f6BbRp6K5ylPPgXHNXwPtrNGpFYWh7nOtJ8PmsRnx',
    'group.id': 'my_group',  # Set a consumer group ID
    'auto.offset.reset': 'earliest' # Start consuming from the beginning of the topic
}

consumer = Consumer(conf)
consumer.subscribe(["topic_0"]) # Replace with your topic name

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        print(f"Received message: {msg.value().decode('utf-8')}")

except KeyboardInterrupt:
    pass

finally:
    consumer.close()


Received message: {'customer_id': '18', 'name': 'Customer_18', 'city': 'Pune', 'state': 'Delhi', 'country': 'India', 'registration_date': '2023-10-04', 'is_active': 'True'}
Received message: {'customer_id': '20', 'name': 'Customer_20', 'city': 'Hyderabad', 'state': 'Telangana', 'country': 'India', 'registration_date': '2023-01-01', 'is_active': 'True'}
Received message: {'customer_id': '26', 'name': 'Customer_26', 'city': 'Delhi', 'state': 'Delhi', 'country': 'India', 'registration_date': '2023-03-22', 'is_active': 'True'}
Received message: {'customer_id': '36', 'name': 'Customer_36', 'city': 'Ahmedabad', 'state': 'Tamil Nadu', 'country': 'India', 'registration_date': '2023-10-07', 'is_active': 'True'}
Received message: {'customer_id': '42', 'name': 'Customer_42', 'city': 'Bangalore', 'state': 'Gujarat', 'country': 'India', 'registration_date': '2023-08-11', 'is_active': 'True'}
Received message: {'customer_id': '43', 'name': 'Customer_43', 'city': 'Hyderabad', 'state': 'Tamil Nadu', '

In [ ]:
# prompt: Give me the kafka code to send the records in first_100_customers.csv to my kafka topic

from confluent_kafka import Producer
import csv

# Assuming 'first_100_customers.csv' is in the current directory
csv_file_path = 'first_100_customers.csv'

conf = {
    'bootstrap.servers': 'pkc-619z3.us-east1.gcp.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'BVDQLUUYCEQ2B36K',
    'sasl.password': 'wKsvL2/e4Vu260owTCH8+z2f6BbRp6K5ylPPgXHNXwPtrNGpFYWh7nOtJ8PmsRnx',
}

producer = Producer(conf)

def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

topic = "topic_0"  # Replace with your topic name

try:
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)  # Assuming the CSV has a header row
        for row in reader:
            # Convert the row (dictionary) to a string or JSON
            message = str(row) # Or json.dumps(row) if you prefer JSON format
            producer.produce(topic, message.encode('utf-8'), callback=delivery_report)
            producer.poll(0)  # Handle delivery reports

    producer.flush()
    print("All messages sent successfully.")

except FileNotFoundError:
    print(f"Error: File '{csv_file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [2]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message delivered to topic_0 [0]
Message de

In [ ]:
#send videos via kafka